## Using Local Ollama model
In past we used Open AI model which is paid one. You can try it with 5$ free tier. The limitations is number of token and API rate limit. 

Alternatively you can install Ollama, pull ollama model and use it locally. 

In this session, we will explore using Ollama model.

Before that, download and install ollama by following https://ollama.com/download

Ollama models: https://ollama.com/search 

- gpt-oss: https://ollama.com/library/gpt-oss 
- llama3.2: https://ollama.com/library/llama3.2 
- deepseek-r1: https://ollama.com/library/deepseek-r1

Pull below models by 

ollama pull [model name] or

ollama run [model name]

- llama3.2:latest
- gpt-oss:latest
- deepseek-r1:latest




#### Import libraries

In [ ]:
# To use local ollama model, we need to import AsyncOpenAI and OpenAIChatCompletionsModel
from openai import AsyncOpenAI
from agents import Agent, Runner, OpenAIChatCompletionsModel, SQLiteSession
from dotenv import load_dotenv

#### Load openai api key from environment

In [ ]:
# Load environment variables
load_dotenv(override=True)

#### Define OpenAI client and llm model

In [ ]:
# Define the OpenAI client pointing to a local LLM server
client = AsyncOpenAI(base_url="http://localhost:11434/v1")

# Define model using client
model_name = "llama3.2:latest"
# model_name = "gpt-oss"

# gpt-oss is a thinking model. at least 13GB free RAM is required to run it locally. 
model = OpenAIChatCompletionsModel(model = model_name,openai_client= client)

#### Define Agent

In [ ]:
# Create an agent using the local model
local_agent = Agent(name="Bioinfo Local Agent",
                    instructions="You are a helpful bioinformatics assistant using local LLM server.",
                    model=model)

#### Run Agent as chat interface with session enabled

In [ ]:
session = SQLiteSession("bioinfo_session")

async def chat(message, history):
    result = await Runner.run(local_agent, message, session=session)
    return result.final_output


import gradio as gr
gr.ChatInterface(
    chat,
    title="Bioinformatics Expert Chatbot",
    description="Chat with the Bioinformatics Expert"
).launch()


## Use models of other LLM providers

The process is same as using Local LLM as shown above. Only urls are different. 

We must have API key setup for it work and you may be in their paid plan.

In [ ]:
# Define base URLs for Gemini and Deepseek APIs.
# These urls are compatible with OpenAI API endpoints.
# So openAI sdk agent can be used to call these models
GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
DEEPSEEK_BASE_URL = "https://api.deepseek.com/v1"

from openai import AsyncOpenAI
from agents import OpenAIChatCompletionsModel
import os

# Initialize AsyncOpenAI clients for Deepseek and Gemini using the base URLs
# and the respective API keys from environment variables.
# Client helps to make calls to the models
deepseek_client = AsyncOpenAI(base_url=DEEPSEEK_BASE_URL, api_key=os.getenv("DEEPSEEK_API_KEY"))
gemini_client = AsyncOpenAI(base_url=GEMINI_BASE_URL, api_key=os.getenv("GOOGLE_API_KEY"))

# Create OpenAIChatCompletionsModel instances for Deepseek and Gemini models
# OpenAIChatCompletionsModel allows using these models with the agent framework
deepseek_model = OpenAIChatCompletionsModel(model="deepseek-chat", openai_client=deepseek_client)
gemini_model = OpenAIChatCompletionsModel(model="gemini-2.0-flash", openai_client=gemini_client)

instructions = """You are an expert in Bioinformatics domain.
Provide detailed and accurate explanations to user queries."""

deepseek_agent = Agent(name="Explainer",
              instructions=instructions,
                model=deepseek_model)

gemini_agent = Agent(name="Explainer",
              instructions=instructions,
                model=gemini_model)

query = "Tell me about ACMG classification of genetic variants"

result_deepseek = await Runner.run(deepseek_agent, query)
result_gemini = await Runner.run(gemini_agent, query)

print("Deepseek Agent Response:\n", result_deepseek.final_output)
print("\nGemini Agent Response:\n", result_gemini.final_output)